# Tutorial for sampling using Cirq
In this tutorial, we will look at tutorials of implementations using **Cirq**. Participants are recommended to read the `sampling.ipynb` beforehand.

In QURI Parts, there are codes to convert **Cirq** circuits and operators to **QURI Parts**. When implementing with **Cirq**, you can use these codes to use the provided sampling function with cirq circuits and operators. Let's actually implement it.

## ChallengeSampling

In [18]:
import sys

sys.path.append("../")
from utils.challenge_2023 import ChallengeSampling

# define challenge_sampling with or without noise
challenge_sampling = ChallengeSampling(noise=True)

# Sampler

### Prepare a Cirq circuit

In [19]:
from cirq.circuits.circuit import Circuit as CirqCircuit
from cirq.devices.line_qubit import LineQubit
from cirq.ops.common_gates import CNOT, H, Rz
from cirq.ops.pauli_gates import X, Y
from math import pi

cirq_circuit = CirqCircuit()
q0 = LineQubit(0)
q1 = LineQubit(1)
q2 = LineQubit(2)
q3 = LineQubit(3)

cirq_circuit.append(
    [
        X(q0),
        H(q1),
        Y(q2),
        CNOT(q1, q2),
        Rz(rads=pi/4).on(q3),
    ]
)

You can use `circuit_from_cirq()` to convert the cirq circuit to the quri-parts circuit as follows:

In [20]:
from quri_parts.cirq.circuit.cirq_circuit_converter import circuit_from_cirq

quri_parts_circuit = circuit_from_cirq(cirq_circuit)

Then you can use the sampler in the same way as in quri-parts case.

In [21]:
# choose hardware type
hardware_type = "sc"

# create sampler
sampler = challenge_sampling.create_sampler(hardware_type)
# possible to choose Qiksit circuits as an input
sampling_result = sampler(quri_parts_circuit, n_shots=1000)
print(f"counts: {sampling_result}")

counts: Counter({5: 492, 3: 441, 1: 22, 7: 18, 11: 9, 4: 7, 13: 6, 2: 5})


# Sampling Estimator
In order to estimate an expectation value of operators involves operators and states, you need to use a sampling_estimator. You can use `operator_from_cirq_op()` to convert the cirq operator to the quri-parts operator as follows:

### Prepare a Cirq operator

In [22]:
from cirq.ops.linear_combinations import PauliSum
from cirq.ops.pauli_gates import X, Y, Z
from cirq.ops.pauli_string import PauliString

cirq_operator = PauliSum.from_pauli_strings(
        [
            PauliString(0.25 + 1.22j, Z(q0)),
            PauliString(2.0, Z(q1), Z(q2)),
            PauliString(0.5 + 0.25j, X(q0), X(q3)),
        ]
    )
print(cirq_operator)

(0.250+1.220j)*Z(q(0))+2.000*Z(q(1))*Z(q(2))+(0.500+0.250j)*X(q(0))*X(q(3))


You can use `operator_from_cirq_op()` to convert the cirq operator to the quri-parts operator as follows:

In [23]:
from quri_parts.cirq.operator import operator_from_cirq_op

quri_parts_operator = operator_from_cirq_op(cirq_operator)

Then you can use the sampling_estimator in the same way as in quri-parts case.

In [24]:
from quri_parts.core.measurement import bitwise_commuting_pauli_measurement
from quri_parts.core.sampling.shots_allocator import create_equipartition_shots_allocator
from quri_parts.core.state import GeneralCircuitQuantumState

shots_allocator = create_equipartition_shots_allocator()
measurement_factory = bitwise_commuting_pauli_measurement

circuit_state = GeneralCircuitQuantumState(4, quri_parts_circuit)

# returns estimated value using above inputs
estimated_value = challenge_sampling.sampling_estimator(
    operator=quri_parts_operator,
    state_or_circuit=circuit_state,
    n_shots=10000,
    measurement_factory=measurement_factory,
    shots_allocator=shots_allocator,
    hardware_type="sc",
)

# returns real part of estimated value
print(f"estimated_value :{estimated_value.value.real} ")

estimated_value :-2.1091 
